In [1]:
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
sys.path.insert(0,"../")
import mplhep as hep
import pickle
import glob
import ROOT as rt
import coffea
import awkward as ak
from coffea import hist, processor
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
import pandas as pd

Welcome to JupyROOT 6.28/00


In [2]:
sys.path.append("/uscms/home/amalbert/nobackup/CMSSW_14_1_0_pre4/src/RazorCommon/Tools/bin")
import importlib
import getMuonScaleFactor

In [3]:
#paths
MC_path = "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/MC_noClusters/MC_Summer23/DYto2Mu_MLL-50to120_Merged/DYto2Mu_MLL-50to120_Merged_18666pb_weighted.root"

data_path_list = ["/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023B_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023B_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v1_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v2_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v2_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v3_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v3_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon0_Run2023C_PromptReco-v4_goodLumi.root",
                 "/store/group/lpclonglived/amalbert/Data_MC_Comp_TnP/results_from_cache_noSkim/Data_noClusters/2023_Merged/Muon1_Run2023C_PromptReco-v4_goodLumi.root"]

In [4]:
ak.behavior.update(candidate.behavior)

def getLZDF(f,nEvents=-1,version="new"): #lazy dataframe with events that have cluster matched to probe muon
    events_raw = uproot.open(f)['MuonSystem']
    df = coffea.processor.LazyDataFrame(events_raw,entrystop=nEvents)
    start,stop = df._branchargs['entry_start'],df._branchargs['entry_stop']
    events = uproot.lazy(df._tree)
    #events = events[start:stop]
    return events

In [28]:
#events_data = getLZDF(data_path)
events_MC_full = getLZDF("root://cmseos.fnal.gov/"+MC_path)

events_MC_full = events_MC_full[np.logical_and(events_MC_full.Flag_all, events_MC_full.Flag_ecalBadCalibFilter)]
events_MC_full = events_MC_full[events_MC_full.jetVeto]

In [29]:
data_events = [getLZDF("root://cmseos.fnal.gov/"+data_path) for data_path in data_path_list]

In [30]:
events_data_full = ak.concatenate(data_events, axis=0)

#events_data_full = events_data_full[events_data_full.nCscRechitClusters>0]
events_data_full = events_data_full[np.logical_and(events_data_full.ZMass>50, events_data_full.ZMass<120)]
events_data_full = events_data_full[np.logical_and(events_data_full.Flag_all, events_data_full.Flag_ecalBadCalibFilter)]
events_data_full = events_data_full[events_data_full.jetVeto]

In [31]:
events_data_full["lepTag"]
lepTag_array = events_data_full["lepTag"]
lepTag_array = np.repeat(lepTag_array,np.array(events_data_full["numTag"]), axis=0)
arr = np.zeros(np.shape(lepTag_array)[0], dtype=bool)
arr[::2] = 1  # Set even indices to 1
mask = ak.all(lepTag_array, axis=1)
print(lepTag_array)
#print(mask)
mask_T_F = np.logical_and(arr, mask)
mask_F_T = np.logical_and(np.logical_not(arr), mask)

#indices_T_F = np.zip()
lepTag_array = np.array(lepTag_array)
lepTag_array.reshape((np.array(mask_T_F).shape[0],2))
#print(mask_T_F)
#print(mask_F_T)
lepTag_array[mask_T_F] = [True, False]
lepTag_array[mask_F_T] = [False, True]
print(lepTag_array)

[[False, True], [True, True], [True, True, ... True], [True, True], [True, True]]
[[False  True]
 [False  True]
 [ True False]
 ...
 [False  True]
 [ True False]
 [False  True]]


In [32]:
branch_names = ["weight", "pileupWeight", "ZMass", "met", "metPhi", "puppiMet", "puppiMetPhi","Flag_all", "Flag_ecalBadCalibFilter", "jetVeto"]

In [33]:
def getBranches(LZDF, isMC=False):
    new_df = ak.zip({field: np.repeat(LZDF[field], LZDF["numTag"]) for field in branch_names})
#     for branch in branch_names:
#         if (not isMC) and (branch in ["weight", "pileupWeight"]):
#             continue
#         new_df = ak.with_field(new_df, np.repeat(LZDF[branch],LZDF["numTag"]), branch)
    print("finished event level variables")
    
    lepTag_array = LZDF["lepTag"]
    lepTag_array = np.repeat(lepTag_array,np.array(LZDF["numTag"]), axis=0)
    arr = np.zeros(np.shape(lepTag_array)[0], dtype=bool)
    arr[::2] = 1  # Set even indices to 1
    mask = ak.all(lepTag_array, axis=1)
    mask_T_F = np.logical_and(arr, mask)
    mask_F_T = np.logical_and(np.logical_not(arr), mask)
    lepTag_array = np.array(lepTag_array)
    lepTag_array.reshape((np.array(mask_T_F).shape[0],2))
    lepTag_array[mask_T_F] = [True, False]
    lepTag_array[mask_F_T] = [False, True]
    
    row_indices = np.arange(lepTag_array.shape[0], dtype=int)
    column_indices_probe =np.where(lepTag_array==False)[1]
    column_indices_tag =np.where(lepTag_array==True)[1]
    
    
    if isMC:
        MC_SF_LooseID = getMuonScaleFactor.getLooseIDEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_LooseISO = getMuonScaleFactor.getLooseISOEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_TightID = getMuonScaleFactor.getTightIDEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_TightISO = getMuonScaleFactor.getTightISOEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        MC_SF_HLT = getMuonScaleFactor.getHLTEffArr_preBPix(np.array(LZDF.lepPt), np.array(LZDF.lepEta))
        
        MC_SF_LooseID = np.repeat(MC_SF_LooseID,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_LooseISO = np.repeat(np.array(MC_SF_LooseISO),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
        MC_SF_TightID = np.repeat(MC_SF_TightID,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_TightISO = np.repeat(MC_SF_TightISO,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        MC_SF_HLT = np.repeat(MC_SF_HLT,np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
        
        MC_Weight_Total = new_df["weight"]*MC_SF_LooseID*MC_SF_LooseISO*MC_SF_TightID*MC_SF_TightISO*MC_SF_HLT
        new_df = ak.with_field(new_df, MC_Weight_Total, "weight_total")
    
    print("at muon variables")
    #load pT, eta, and phi for tag and probe muons
    probe_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    probe_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    probe_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_probe]
    
    tag_pT = np.repeat(np.array(LZDF["lepPt"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    tag_eta = np.repeat(np.array(LZDF["lepEta"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    tag_phi = np.repeat(np.array(LZDF["lepPhi"]),np.array(LZDF["numTag"]), axis=0)[row_indices,column_indices_tag]
    
    new_df = ak.with_field(new_df, probe_pT, "probe_pT")
    new_df = ak.with_field(new_df, probe_eta, "probe_eta")
    new_df = ak.with_field(new_df, probe_phi, "probe_phi")
    
    new_df = ak.with_field(new_df, tag_pT, "tag_pT")
    new_df = ak.with_field(new_df, tag_eta, "tag_eta")
    new_df = ak.with_field(new_df, tag_phi, "tag_phi")
    
    return new_df

In [ ]:
print("MC")
events_MC = getBranches(events_MC_full, True)
print("now data")
events_data = getBranches(events_data_full, False)

MC
finished event level variables
at muon variables
now data


In [14]:
rt.gStyle.SetOptStat(0)
def make_ratio_plot(h_list_in, title = "", label = "", fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=False, scales = [1,1]):
    h_list = []
    if in_tags == None:
        tag = []
    else:
        tag = in_tags
    for i, h in enumerate(h_list_in):
        h_list.append(h.Clone('h{}aux{}'.format(i, label)))
        if in_tags == None:
            tag.append(h.GetTitle())
    #print("tags: ", tag)
    c_out = rt.TCanvas("c_out_ratio"+label, "c_out_ratio"+label, 800, 800)
    pad1 = rt.TPad("pad1", "pad1", 0, 0.3, 1, 1.0)
    pad1.SetBottomMargin(0.03)
    pad1.SetLeftMargin(0.15)
    pad1.SetRightMargin(0.04)# pad2.SetGrid()
    if logy:
        pad1.SetLogy()

    pad1.Draw()
    pad1.cd()

    leg = rt.TLegend(0.5, 0.65, 0.9, 0.92)
    leg = rt.TLegend(0.7, 0.65, 0.9, 0.92)

    #leg = rt.TLegend(0.2, 0.7, 0.5, 0.9)
    # leg = rt.TLegend(0.7, 0.2, 0.9, 0.4)
    leg.SetBorderSize(0)
    leg.SetTextSize(0.045)
    leg.SetFillStyle(0)
    c_out.cd(1)

    scaled_h_list = []
    if scale:
        for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #h = h_unscaled.Scale(1/scales[i])
            #scaled_h_list.append(h_unscaled.Clone())
            h_unscaled.Scale(1/scales[i])
            scaled_h_list.append(h_unscaled)
    else:
        #for i, h_unscaled in enumerate(h_list):
            #h = h_unscaled.Clone()
            #scaled_h_list.append(h)
        scaled_h_list = h_list
    for i, h in enumerate(scaled_h_list):
        h.GetXaxis().SetLabelSize(0)
        h.GetXaxis().SetTitle(label)
        h.GetYaxis().SetRangeUser(0, 1.1*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and not scale:
            h.GetYaxis().SetRangeUser(10e-2, 2*max(map(lambda x: x.GetMaximum(), scaled_h_list)))
        if logy and scale:
            h.GetYaxis().SetRangeUser(10e-4, 1)
        h.GetYaxis().SetTitleOffset(1.0)
        h.GetYaxis().SetTitleSize(0.06)
        h.GetYaxis().SetLabelSize(0.05)
        
        if scale:
            y_title = "Fraction of Events"
        else:
            y_title = "Events"
        
        h.GetYaxis().SetTitle()
        h.SetTitle(f"{title};adsf;{y_title}")
        #if ratio_index == 0:h.DrawCopy("hist")
        '''
        h.SetFillColor(h_list_in[i].GetLineColor())
        h.SetFillStyle(3002)
        #h.SetStats(1)
        h.SetLineColor(h_list_in[i].GetLineColor())
        h.SetLineWidth(2)
        h.SetMarkerColor(h_list_in[i].GetLineColor())
        h.SetMarkerSize(2)
        # if ratio_index == 0:
        #     # h.DrawCopy("hist")
        #     h.DrawCopy(draw_opt[i]+'same')
        # else:h.DrawCopy(draw_opt[i])
        #if ratio_index == 0 :h.DrawCopy(draw_opt[i]+"same")
        #h.DrawCopy("E2 HIST")
        '''
        if i==0:
            h.SetLineWidth(4)
            h.DrawCopy("hist")
            #h.SetFillStyle(0)
            h.SetFillColor(h_list_in[i].GetLineColor())
            h.SetFillStyle(3002)
            #h.SetStats(1)
            h.SetLineColor(h_list_in[i].GetLineColor())
            h.SetLineWidth(2)
            h.SetMarkerColor(h_list_in[i].GetLineColor())
            h.SetMarkerSize(2)
            h.DrawCopy(draw_opt[i] + "same")
            #h.SetFillStyle(0)
        else:
            h.SetLineWidth(2)
            h.DrawCopy(draw_opt[i] + "same")
        #else:h.DrawCopy(draw_opt[i])
        if len(text)>0:
            l = rt.TLatex()
            l.SetTextSize(0.045)
            if logy:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()/10, text)
            else:l.DrawLatex((h.GetXaxis().GetXmax()-h.GetXaxis().GetXmin())*0.1+h.GetXaxis().GetXmin() , h.GetMaximum()*0.8, text)
        #if i==1:
            #h.DrawCopy(draw_opt[i]+"same")
       #     h.Draw("E1 same")

        leg.AddEntry(h, tag[i], "lep")
    leg.Draw("same")

    c_out.cd()
    pad2 = rt.TPad("pad2", "pad2", 0, 0, 1, 0.3)
    pad2.SetTopMargin(0.03)
    pad2.SetBottomMargin(0.25)
    pad2.SetLeftMargin(0.15)
    pad2.SetRightMargin(0.04)# pad2.SetGrid()
    pad2.Draw()
    pad2.cd()
    band = scaled_h_list[ratio_index].Clone('h_band')
    for j in range(band.GetXaxis().GetNbins()):
        band.SetBinContent(j+1, 1.0)
        if h_list[ratio_index].GetBinContent(j+1) == 0:
            band.SetBinError(j+1, 0.0)
        else:
            band.SetBinError(j+1, scaled_h_list[ratio_index].GetBinError(j+1)/scaled_h_list[ratio_index].GetBinContent(j+1))
            #print(j, h_list_in[0].GetBinError(j+1)/h_list_in[0].GetBinContent(j+1))
    band.SetFillColor(scaled_h_list[ratio_index].GetLineColor())

    band.SetFillStyle(3002)
    band.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    #band.SetFillColorAlpha(0,0)
    #band.SetLineColor(0)
    
    band.GetYaxis().SetTitleOffset(0.5)
    band.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
    band.GetYaxis().SetTitleSize(0.11)
    band.GetYaxis().SetLabelSize(0.12)
    band.GetYaxis().SetNdivisions(506)
    band.GetXaxis().SetTitleOffset(0.95)
    band.GetXaxis().SetTitleSize(0.12)
    band.GetXaxis().SetLabelSize(0.12)
    band.GetXaxis().SetTickSize(0.07)
    
    band.SetYTitle('Ratio with {}'.format(tag[ratio_index]))
    band.SetXTitle(label)
    band.SetTitle("")
    band.DrawCopy('E2')
    ln = rt.TLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
    ln.SetLineWidth(3)
    ln.SetLineColor(scaled_h_list[ratio_index].GetLineColor())
    ln.DrawLine(h.GetXaxis().GetXmin(), 1, h.GetXaxis().GetXmax(), 1)
     
    #print(ratio_index)
    for i, h in enumerate(scaled_h_list):
        if i == ratio_index:
            continue
        else:
            if fit:h.GetFunction("expo")
            h.Divide(scaled_h_list[ratio_index])
            # h.GetYaxis().SetTitleOffset(0.6)
            # h.GetYaxis().SetRangeUser(ratio_bounds[0], ratio_bounds[1])
            # h.GetYaxis().SetTitleSize(0.12)
            # h.GetYaxis().SetLabelSize(0.12)
            # h.GetYaxis().SetNdivisions(506)
            # h.GetXaxis().SetTitleOffset(0.95)
            # h.GetXaxis().SetTitleSize(0.12)
            # h.GetXaxis().SetLabelSize(0.12)
            # h.GetXaxis().SetTickSize(0.07)
            # h.SetYTitle('Ratio with {}'.format(tag[0]))
            # h.SetTitle("")
            #set relative error of ratio to be the relative error of data
            for j in range(h.GetXaxis().GetNbins()):
                if h_list[i].GetBinContent(j+1) == 0:
                    h.SetBinError(j+1, 0.0)
                else:
                    h.SetBinError(j+1, h_list_in[i].GetBinError(j+1)/h_list_in[i].GetBinContent(j+1)*h.GetBinContent(j+1))
            h.Draw('same'+draw_opt[i])
    
    pad2.Update()
    
    c_out.pad1 = pad1
    c_out.pad2 = pad2
    c_out.h_list = h_list
    c_out.leg = leg
    
    return c_out

In [15]:
import os
plot_info = {}
plot_output = "Run2023_preBPix_Data_MC_Comp_noClusters_noiseFilters"
os.makedirs(plot_output, exist_ok=True)

In [19]:
branch = "ZMass"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=80, xlow = 0, xup=150)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=80, xlow = 0, xup=150)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)                  
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Z Candidate Mass", "label": "Z Candidate Mass [GeV]", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).


In [20]:
branch = "probe_pT"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=100)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)
print(data_size)
MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=100)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
print(sumOfWeights)
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Probe pT", "label": "pT [GeV]", "scales": [sumOfWeights, data_size], "logy": False}

21053829
23552404.557402298


Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [21]:
branch = "tag_pT"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=25, xlow = 0, xup=100)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
data.FillN(data_size, data_arr, data_weights)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=25, xlow = 0, xup=100)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Tag pT", "label": "pT [GeV]", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [22]:
for plot_type, plot_info_dict in plot_info.items():
    print(plot_type)
    print(plot_info_dict)
    for boolScale in [True, False]:
        c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        if boolScale:
            scaleString = "_normalized"
            #c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2 hist','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        else:
            scaleString=""
        os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
        c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+"_updatedTNP.png")
        

ZMass
{'MC_hist': <cppyy.gbl.TH1F object at 0xed1e710>, 'data_hist': <cppyy.gbl.TH1F object at 0x3013b1c0>, 'file_name': 'ZMass', 'title': 'Z Candidate Mass', 'label': 'Z Candidate Mass [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
probe_pT
{'MC_hist': <cppyy.gbl.TH1F object at 0x15571f80>, 'data_hist': <cppyy.gbl.TH1F object at 0x3cd6f240>, 'file_name': 'probe_pT', 'title': 'Probe pT', 'label': 'pT [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
tag_pT
{'MC_hist': <cppyy.gbl.TH1F object at 0x6a38820>, 'data_hist': <cppyy.gbl.TH1F object at 0x3cd69ad0>, 'file_name': 'tag_pT', 'title': 'Tag pT', 'label': 'pT [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}


Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/ZMass/ZMass_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioZ Candidate Mass [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/ZMass/ZMass_updatedTNP.png has been created
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/probe_pT/probe_pT_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratiopT [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/probe_pT/probe_pT_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratiopT [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/tag_pT/tag_pT_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratiopT [G

In [23]:
branch = "probe_eta"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=30, xlow = -4, xup=4)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=30, xlow = -4, xup=4)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Probe Eta", "label": "Eta", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [24]:
branch = "tag_eta"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=30, xlow = -4, xup=4)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=30, xlow = -4, xup=4)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Tag Eta", "label": "Eta", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [25]:
branch = "probe_phi"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -4, xup=4)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -4, xup=4)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Probe Phi", "label": "Phi", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [26]:
for plot_type, plot_info_dict in plot_info.items():
    print(plot_type)
    print(plot_info_dict)
    for boolScale in [True, False]:
        c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        if boolScale:
            scaleString = "_normalized"
            #c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2 hist','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        else:
            scaleString=""
        os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
        c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+"_updatedTNP.png")

ZMass
{'MC_hist': <cppyy.gbl.TH1F object at 0xed1e710>, 'data_hist': <cppyy.gbl.TH1F object at 0x3013b1c0>, 'file_name': 'ZMass', 'title': 'Z Candidate Mass', 'label': 'Z Candidate Mass [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
probe_pT
{'MC_hist': <cppyy.gbl.TH1F object at 0x15571f80>, 'data_hist': <cppyy.gbl.TH1F object at 0x3cd6f240>, 'file_name': 'probe_pT', 'title': 'Probe pT', 'label': 'pT [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
tag_pT
{'MC_hist': <cppyy.gbl.TH1F object at 0x6a38820>, 'data_hist': <cppyy.gbl.TH1F object at 0x3cd69ad0>, 'file_name': 'tag_pT', 'title': 'Tag pT', 'label': 'pT [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
probe_eta
{'MC_hist': <cppyy.gbl.TH1F object at 0x141ccff0>, 'data_hist': <cppyy.gbl.TH1F object at 0x15569fd0>, 'file_name': 'probe_eta', 'title': 'Probe Eta', 'label': 'Eta', 'scales': [23552404.557402298, 21053829], 'logy': False}
tag_eta
{'MC_hist': <cppyy.gbl.TH1F object at 0x3a

Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/ZMass/ZMass_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioZ Candidate Mass [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/ZMass/ZMass_updatedTNP.png has been created
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/probe_pT/probe_pT_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratiopT [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/probe_pT/probe_pT_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratiopT [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/tag_pT/tag_pT_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratiopT [G

In [18]:
branch = "tag_phi"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -4, xup=4)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -4, xup=4)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "Tag Phi", "label": "Phi", "scales": [sumOfWeights, data_size], "logy": False}

In [19]:
branch = "met"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -5, xup=200)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -5, xup=200)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "MET", "label": "MET [GeV]", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [20]:
branch = "puppiMet"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=60, xlow = -5, xup=200)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=60, xlow = -5, xup=200)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "PUPPI MET", "label": "PUPPI MET [GeV]", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [21]:
branch = "metPhi"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=30, xlow = -4, xup=4)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=30, xlow = -4, xup=4)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "MET Phi", "label": "MET Phi", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [22]:
branch = "puppiMetPhi"
name = branch
data = rt.TH1F("LeadingPt_Data", "Data", nbinsx=30, xlow = -4, xup=4)

#mask = makeEventMask(events_data, ["forward_veto"], True)
#data_tree = events_data[mask]
#data_tree = data_tree[~ak.is_none(data_tree)]
data_tree = events_data
# for num, entry in enumerate(data_tree):
#     if num%100000==0:print(num)
#     data.Fill(entry[branch])
data_arr = np.array(data_tree[branch], dtype=np.float64)
data_size = np.size(data_arr)
data_weights = np.ones(data_size, dtype=np.float64)
#data.FillN(np.shape(data_tree[branch])[0],np.array(data_tree[branch]), None,1)
data.FillN(data_size, data_arr, data_weights)
data.SetLineColor(rt.kBlack)
data.SetFillStyle(0)

MC = rt.TH1F("LeadingPt_MC", "MC", nbinsx=30, xlow = -4, xup=4)
#mask = makeEventMask(events_MC, ["forward_veto"], True)
#clusterIndex=-1
sumOfWeights=0
#MC_tree = events_MC[mask]
#MC_tree = MC_tree[~ak.is_none(MC_tree)]
MC_tree = events_MC
# for num,entry in enumerate(MC_tree):
#     if num%100000==0:print(num)
#     MC.Fill(entry[branch], entry["weight_total"])
#     sumOfWeights+=entry["weight_total"]
#MC.FillN(np.size(MC_tree[branch]),np.array(MC_tree[branch]), MC_tree["weight_total"],1)
#sumOfWeights = ak.sum(MC_tree["weight_total"])
sumOfWeights = ak.sum(MC_tree["weight_total"])
MC_arr = np.array(MC_tree[branch], dtype=np.float64)
MC_size = np.size(MC_arr)
MC_weights = np.array(MC_tree["weight_total"])
MC.FillN(MC_size, MC_arr, MC_weights)
MC.SetLineColor(rt.kRed)
MC.SetFillStyle(0)
plot_info[name] = {"MC_hist": MC, "data_hist": data, "file_name": name, "title": "PUPPI MET Phi", "label": "PUPPI MET Phi", "scales": [sumOfWeights, data_size], "logy": False}

Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_Data (Potential memory leak).
Warning in <TROOT::Append>: Replacing existing TH1: LeadingPt_MC (Potential memory leak).


In [23]:
for plot_type, plot_info_dict in plot_info.items():
    print(plot_type)
    print(plot_info_dict)
    for boolScale in [True, False]:
        c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = plot_info_dict["logy"], ratio_index = 0, draw_opt = ['E2','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        if boolScale:
            scaleString = "_normalized"
            #c = make_ratio_plot([plot_info_dict["MC_hist"], plot_info_dict["data_hist"]], title = plot_info_dict["title"], label = plot_info_dict["label"], fit = False, in_tags = None, ratio_bounds = [0.1, 4], logy = False, ratio_index = 0, draw_opt = ['E2 hist','E1'], text = "", scale=boolScale, scales = plot_info_dict["scales"])
        else:
            scaleString=""
        os.makedirs(plot_output+"/"+plot_info_dict["file_name"], exist_ok=True)
        c.SaveAs(plot_output+"/"+plot_info_dict["file_name"]+"/"+plot_info_dict["file_name"]+scaleString+"_updatedTNP.png")
        

tag_phi
{'MC_hist': <cppyy.gbl.TH1F object at 0x43ed1c0>, 'data_hist': <cppyy.gbl.TH1F object at 0x122917f0>, 'file_name': 'tag_phi', 'title': 'Tag Phi', 'label': 'Phi', 'scales': [23552404.557402298, 21053829], 'logy': False}
met
{'MC_hist': <cppyy.gbl.TH1F object at 0x1587a7c0>, 'data_hist': <cppyy.gbl.TH1F object at 0x16055f60>, 'file_name': 'met', 'title': 'MET', 'label': 'MET [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
puppiMet
{'MC_hist': <cppyy.gbl.TH1F object at 0x15139b60>, 'data_hist': <cppyy.gbl.TH1F object at 0x18dfc140>, 'file_name': 'puppiMet', 'title': 'PUPPI MET', 'label': 'PUPPI MET [GeV]', 'scales': [23552404.557402298, 21053829], 'logy': False}
metPhi
{'MC_hist': <cppyy.gbl.TH1F object at 0x27e07960>, 'data_hist': <cppyy.gbl.TH1F object at 0x18df7b40>, 'file_name': 'metPhi', 'title': 'MET Phi', 'label': 'MET Phi', 'scales': [23552404.557402298, 21053829], 'logy': False}
puppiMetPhi
{'MC_hist': <cppyy.gbl.TH1F object at 0x1cd67490>, 'data_hist': <

Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/tag_phi/tag_phi_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioPhi
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/tag_phi/tag_phi_updatedTNP.png has been created
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/met/met_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioMET [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/met/met_updatedTNP.png has been created
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/puppiMet/puppiMet_normalized_updatedTNP.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: c_out_ratioPUPPI MET [GeV]
Info in <TCanvas::Print>: png file Run2023_preBPix_Data_MC_Comp_noClusters/puppiMet/puppiMet_updatedTN

In [24]:
print(sumOfWeights)
print(ak.size(events_data.ZMass))

23552404.557402298
21053829
